<a href="https://colab.research.google.com/github/sbarrerab/MetNumUN2023I/blob/main/Week4IterativeMethodsForLinearSystemsGroup4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple iteration for systems of linear equations

First, generate a random diagonally dominant matrix, for testing.

In [1]:
import numpy as np
rndm = np.random.RandomState(1234)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

In [2]:
A

array([[1.51915195e+01, 6.22108771e-01, 4.37727739e-01, 7.85358584e-01,
        7.79975808e-01, 2.72592605e-01, 2.76464255e-01, 8.01872178e-01,
        9.58139354e-01, 8.75932635e-01],
       [3.57817270e-01, 1.55009951e+01, 6.83462935e-01, 7.12702027e-01,
        3.70250755e-01, 5.61196186e-01, 5.03083165e-01, 1.37684496e-02,
        7.72826622e-01, 8.82641191e-01],
       [3.64885984e-01, 6.15396178e-01, 1.50753812e+01, 3.68824006e-01,
        9.33140102e-01, 6.51378143e-01, 3.97202578e-01, 7.88730143e-01,
        3.16836122e-01, 5.68098653e-01],
       [8.69127390e-01, 4.36173424e-01, 8.02147642e-01, 1.51437668e+01,
        7.04260971e-01, 7.04581308e-01, 2.18792106e-01, 9.24867629e-01,
        4.42140755e-01, 9.09315959e-01],
       [5.98092228e-02, 1.84287084e-01, 4.73552788e-02, 6.74880944e-01,
        1.55946248e+01, 5.33310163e-01, 4.33240627e-02, 5.61433080e-01,
        3.29668446e-01, 5.02966833e-01],
       [1.11894318e-01, 6.07193706e-01, 5.65944643e-01, 6.76406199e-03,
   

# I.  Jacobi iteration

Given

$$
A x = b
$$

separate the diagonal part $D$,

$$ A = D + (A - D) $$

and write

$$
x = D^{-1} (D - A) x + D^{-1} b\;.
$$

Then iterate

$$
x_{n + 1} = B x_{n} + c\;,
$$

where 

$$
B = D^{-1} (A - D) \qquad \text{and} \qquad c = D^{-1} b
$$


Let's construct the matrix and the r.h.s. for the Jacobi iteration

In [3]:
diag_1d = np.diag(A)

B = -A.copy()
np.fill_diagonal(B, 0)

D = np.diag(diag_1d)
invD = np.diag(1./diag_1d)
BB = invD @ B 
c = invD @ b

In [4]:
# sanity checks
from numpy.testing import assert_allclose

assert_allclose(-B + D, A)


# xx is a "ground truth" solution, compute it using a direct method
xx = np.linalg.solve(A, b)

np.testing.assert_allclose(A@xx, b)
np.testing.assert_allclose(D@xx, B@xx + b)
np.testing.assert_allclose(xx, BB@xx + c)

Check that $\| B\| \leqslant 1$:

In [5]:
np.linalg.norm(BB)

0.36436161983015336

### Do the Jacobi iteration

In [6]:
n_iter = 50

x0 = np.ones(n)
x = x0
for _ in range(n_iter):
    x = BB @ x + c

In [7]:
# Check the result:

A @ x - b

array([ 1.11022302e-16,  0.00000000e+00, -2.22044605e-16, -1.11022302e-16,
        1.11022302e-16,  0.00000000e+00, -2.42861287e-17,  0.00000000e+00,
       -2.77555756e-17,  1.11022302e-16])

### Task I.1

Collect the proof-of-concept above into a single function implementing the Jacobi iteration. This function should receive the r.h.s. matrix $A$, the l.h.s. vector `b`, and the number of iterations to perform.


The matrix $A$ in the illustration above is strongly diagonally dominant, by construction. 
What happens if the diagonal matrix elements of $A$ are made smaller? Check the convergence of the Jacobi iteration, and check the value of the norm of $B$.

(20% of the total grade)


In [8]:
def jacobi_iteration(A, b, n_iter):
    # Check if the matrix A is square
    if A.shape[0] != A.shape[1]:
        raise ValueError("Matrix A must be square")

    # Check if the dimensions of A and b are compatible
    if A.shape[0] != b.shape[0]:
        raise ValueError("Dimensions of A and b are not compatible")

    # Extract the diagonal elements of A
    diag_1d = np.diag(A)

    # Compute the matrices B and c for the Jacobi iteration
    B = -A.copy()
    np.fill_diagonal(B, 0)

    D = np.diag(diag_1d)
    invD = np.diag(1. / diag_1d)
    BB = invD @ B
    c = invD @ b

    # Check the norm of B
    norm_B = np.linalg.norm(BB)

    # Perform the Jacobi iteration
    x = np.ones(b.shape)
    for _ in range(n_iter):
        x = BB @ x + c

    # Calculate the residual error
    residual = A @ x - b

    return x, residual, norm_B


In [9]:
# Generate a random diagonally dominant matrix
rndm = np.random.RandomState(1234)
n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

# Construct the matrix and the r.h.s. for the Jacobi iteration
diag_1d = np.diag(A)
B = -A.copy()
np.fill_diagonal(B, 0)
D = np.diag(diag_1d)
invD = np.diag(1./diag_1d)
BB = invD @ B
c = invD @ b

# Check the norm of B
norm_B = np.linalg.norm(BB)

# Perform the Jacobi iteration
n_iter = 50
x = np.ones(n)
for _ in range(n_iter):
    x = BB @ x + c

# Check the result
residual = A @ x - b

# Analysis
print("Jacobi Iteration:")
print("Solution:")
print(x)
print("Residual error:")
print(residual)
print("Norm of B:")
print(norm_B)

Jacobi Iteration:
Solution:
[ 0.03919429  0.03780037  0.04283232  0.02365951  0.05745031 -0.00030244
 -0.00577279  0.03177549 -0.00422849  0.05284648]
Residual error:
[ 1.11022302e-16  0.00000000e+00 -2.22044605e-16 -1.11022302e-16
  1.11022302e-16  0.00000000e+00 -2.42861287e-17  0.00000000e+00
 -2.77555756e-17  1.11022302e-16]
Norm of B:
0.36436161983015336


# II. Seidel's iteration.

##### Task II.1

Implement the Seidel's iteration. 

Test it on a random matrix. Study the convergence of iterations, relate to the norm of the iteration matrix.

(30% of the total grade)

In [10]:
def seidel_iteration(A, b, n_iter):
    # Check if the matrix A is square
    if A.shape[0] != A.shape[1]:
        raise ValueError("Matrix A must be square")

    # Check if the dimensions of A and b are compatible
    if A.shape[0] != b.shape[0]:
        raise ValueError("Dimensions of A and b are not compatible")

    # Extract the diagonal and lower triangular parts of A
    diag_1d = np.diag(A)
    L = np.tril(A, k=-1)

    # Compute the matrix B and vector c for the Seidel's iteration
    invDL = np.linalg.inv(np.diag(diag_1d) - L)
    B = -invDL @ L
    c = invDL @ b

    # Check the norm of B
    norm_B = np.linalg.norm(B)

    # Perform the Seidel's iteration
    x = np.zeros(b.shape)
    for _ in range(n_iter):
        x_new = np.zeros(b.shape)
        for i in range(b.shape[0]):
            x_new[i] = B[i] @ x_new + c[i]
        x = x_new

    # Calculate the residual error
    residual = A @ x - b

    return x, residual, norm_B

In [11]:
# Generate a random matrix and a random right-hand side vector
rndm = np.random.RandomState(1234)
n = 10
A = rndm.uniform(size=(n, n))
b = rndm.uniform(size=n)

# Set the number of iterations
n_iter = 50

# Perform the Seidel's iteration
x = np.zeros(b.shape)
for _ in range(n_iter):
    for i in range(b.shape[0]):
        x[i] = (b[i] - np.dot(A[i, :i], x[:i]) - np.dot(A[i, i+1:], x[i+1:])) / A[i, i]

# Check the result
residual = A @ x - b

# Analysis
print("Seidel's Iteration:")
print("Solution:")
print(x)
print("Residual error:")
print(residual)

Seidel's Iteration:
Solution:
[ 3.60745824e+111  2.16710431e+111 -4.19690182e+112  2.01198043e+113
 -2.25635399e+113  1.76097337e+113  1.19031338e+113 -3.02362308e+113
  3.23732514e+113  2.11349413e+112]
Residual error:
[ 1.32838994e+113  4.56933258e+113 -9.87763785e+112 -1.26738800e+113
  4.69137878e+112  1.21896510e+113 -2.55603227e+113  8.40767198e+111
  9.31208599e+111  2.56318444e+097]


# III. Minimum residual scheme

### Task III.1

Implement the $\textit{minimum residual}$ scheme: an explicit non-stationary method, where at each step you select the iteration parameter $\tau_n$ to minimize the residual $\mathbf{r}_{n+1}$ given $\mathbf{r}_n$. Test it on a random matrix, study the convergence to the solution, in terms of the norm of the residual and the deviation from the ground truth solution (which you can obtain using a direct method). Study how the iteration parameter $\tau_n$ changes as iterations progress.

(50% of the grade)

In [12]:
import numpy as np

def minres_scheme(A, b, n_iter):
    # Check if the matrix A is square
    if A.shape[0] != A.shape[1]:
        raise ValueError("Matrix A must be square")

    # Check if the dimensions of A and b are compatible
    if A.shape[0] != b.shape[0]:
        raise ValueError("Dimensions of A and b are not compatible")

    # Initialize variables
    x = np.zeros(b.shape)
    r = b - A @ x
    p = r.copy()
    T_vals = []

    # Perform the MINRES iteration
    for i in range(n_iter):
        Ap = A @ p
        alpha = np.dot(r, r) / np.dot(p, Ap)
        x = x + alpha * p
        r_new = r - alpha * Ap
        beta = np.dot(r_new, r_new) / np.dot(r, r)
        p = r_new + beta * p
        r = r_new

        # Update T value
        T = np.linalg.norm(r) / np.linalg.norm(b)
        T_vals.append(T)

    # Calculate the residual and deviation from the ground truth solution
    residual = A @ x - b
    deviation = np.linalg.norm(x - np.linalg.lstsq(A, b, rcond=None)[0])

    return x, residual, deviation, T_vals

In [13]:
# Generate a random matrix and a random right-hand side vector
rndm = np.random.RandomState(1234)
n = 10
A = rndm.uniform(size=(n, n))
b = rndm.uniform(size=n)

# Set the number of iterations
n_iter = 50

# Perform the MINRES iteration
x, residual, deviation, T_vals = minres_scheme(A, b, n_iter)

# Calculate the ground truth solution using a direct method
ground_truth = np.linalg.lstsq(A, b, rcond=None)[0]

# Analysis
print("Minimum Residual Scheme:")
print("Solution:")
print(x)
print("Residual error:")
print(residual)
print("Deviation from ground truth:")
print(deviation)
print("Iteration parameters (T):")
print(T_vals)

Minimum Residual Scheme:
Solution:
[ 2.93306283e+14  4.03745250e+14  6.07282799e+14 -3.31969949e+14
  1.39020128e+15 -1.00219407e+15 -8.66213030e+14  3.90113863e+14
 -1.22651120e+15  9.11785662e+14]
Residual error:
[ 8.20427755e+14 -1.35527184e+14  1.01628480e+15  1.60145875e+15
  4.24584026e+14 -1.87959155e+14  9.87561841e+14  2.35857911e+14
  8.56802561e+14  8.38160103e+14]
Deviation from ground truth:
2627410217979341.5
Iteration parameters (T):
[0.6443307497565302, 78.98073711558487, 3120.637708077472, 65310.75927656208, 916409.1241259603, 9562644.603155814, 78673070.4199493, 529777257.5166381, 2996774355.5977507, 14512858840.489359, 61045776992.63848, 225561171462.38004, 738720566489.6774, 2159546189342.093, 5670966926157.41, 13543763883246.371, 29578178710012.668, 59745548708194.69, 106153126927032.36, 161568873460175.47, 223902836926894.4, 261835877612631.5, 283208925936702.94, 257376391490074.03, 166665896228105.12, 100424175987008.5, 60864104887432.06, 32786643045048.15, 15957